In [2]:
import requests as r
from fuzzywuzzy import process
import pprint
pp = pprint.PrettyPrinter(indent=2)

/Users/adam/Documents/vectors/venv/lib/python3.11/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [3]:
response = r.get('https://celestrak.org/NORAD/elements/gp.php?GROUP=active&FORMAT=tle', verify=True)
response.status_code


200

In [4]:
TLE_lines =[line.strip() for line in response.text.split('\n')]
TLE_lines

len(TLE_lines)
if TLE_lines[-1] == '':
    TLE_lines.pop()

while len(TLE_lines) % 3 != 0:
    TLE_lines.pop()
TLEs = {TLE_lines[i]: [TLE_lines[i+1], TLE_lines[i+2]] for i in range(0, len(TLE_lines), 3)}
index = list(TLEs.keys())


def find_satellite(name):

    match = process.extractOne(name, index)
    
    return {
        'query': name,
        'match': match[0],
        'score': match[1],
        'tle': TLEs[match[0]]
    }

find_satellite('NORSAT')

{'query': 'NORSAT',
 'match': 'NORSAT 1',
 'score': 95,
 'tle': ['1 42826U 17042B   25014.49553221  .00008953  00000+0  64024-3 0  9992',
  '2 42826  97.4053 193.4589 0009857  69.1836 291.0446 15.04634661408973']}

In [37]:
def parse_tle_line1(line):
    # Parse each field according to the TLE format specification
    data = {
        'line_number': line[0:1],
        'satellite_number': line[2:7],
        'classification': line[7],
        'launch_year': line[9:11],
        'launch_number': line[11:14],
        'launch_piece': line[14:17],
        'epoch_year': line[18:20],
        'epoch': line[20:32],
        'first_derivative': line[33:43],
        'second_derivative': line[45:52],
        'bstar': line[54:61],
        'ephemeris_type': line[62],
        'element_number': line[64:68],
        'checksum': line[68]
    }
    
    return data

def parse_tle_line2(line):
    data = {
        'line_number': line[0:1],
        'satellite_number': line[1:7],
        'inclination': line[7:16],
        'right_ascension': line[16:25],
        'eccentricity': line[25:32],
        'argument_of_perigee': line[32:41],
        'mean_anomaly': line[41:50], 
        'mean_motion': line[50:62],
        'revolution_number': line[62:68],
        'checksum': line[68]
    }
    return data

parse_tle_line2(match['tle'][1])

match = find_satellite('NORSAT')
print(match['match'])
pp.pprint(parse_tle_line1(match['tle'][0]))
pp.pprint(parse_tle_line2(match['tle'][1]))


NORSAT 1
{ 'bstar': '64024-3',
  'checksum': '2',
  'classification': 'U',
  'element_number': ' 999',
  'ephemeris_type': '0',
  'epoch': '014.49553221',
  'epoch_year': '25',
  'first_derivative': ' .00008953',
  'launch_number': '042',
  'launch_piece': 'B  ',
  'launch_year': '17',
  'line_number': '1',
  'satellite_number': '42826',
  'second_derivative': '00000+0'}
{ 'argument_of_perigee': '7  69.183',
  'checksum': '3',
  'eccentricity': ' 000985',
  'inclination': '  97.4053',
  'line_number': '2',
  'mean_anomaly': '6 291.044',
  'mean_motion': '6 15.0463466',
  'revolution_number': '140897',
  'right_ascension': ' 193.4589',
  'satellite_number': ' 42826'}
